In [ ]:
from glob import glob
from pathlib import Path
import show_result 
import IPython.display as ipd
import os
import pandas as pd

## 1. 피클과 동영상 갯수가 동일하고, 원하는 비디오 개수가 맞는지 확인합니다.


In [ ]:

##################### 여기만 수정하시면 됩니다. ###################
root = "../../dataset/pwb"   # !!상대경로로 되어있어야 합니다.
check_csv = "check_pwb.csv"  # 이름을 검수데이터를 의미하는 값으로 변경하세요
#################################################################



mp4s = sorted(glob(f'{root}/clip/*.mp4'))
mp4s = {Path(e).stem:e for e in mp4s}

anchors = sorted(glob(f'{root}/df_info/df_anchor_i/*.pickle'))
anchors = {Path(e).stem: e for e in anchors}

ppys = sorted(glob(f'{root}/preprocess/*/df_fan.pickle'))
ppys = {Path(e).parent.name:Path(e).parent for e in ppys}

print(f'anchors:{len(anchors)}, mp4s:{len(mp4s)}, ppys:{len(ppys)}')

## 2. error 개수가 0 인 것을 확인합니다.

In [ ]:
# 프리프로세싱되지 않은 pickle list 
errors = set(anchors.keys()) - set(ppys.keys()) 
errors = sorted(list(errors))
print(len(errors))

## 3. 프리프로세싱되지 않은 pickle list 하나하나 살펴보기 

In [ ]:
# 프리프로세싱되지 않은 pickle list 하나하나 살펴보기 
for name in errors:
    print(name)
    df = pd.read_pickle(anchors[name])
    display(df.head())
    
    print('mp4:', mp4s[name[:-4]])
    show_result.display_video(mp4s[name[:-4]])
    
    print('name:', name)
    assert(Path(mp4s[name[:-4]]).exists())
    if input() == 'q':
        break
    ipd.clear_output(wait=True)

## 4. 문제 없는지 일일이 보면서 검사하기 전체 비디오 검사하기
1. 입과 소리가 잘 맞는지 확인한다.
2. 비디오가 이상한 부분이 없는지 확인한다.
3. 특징점이 얼굴위치에 잘 있는지 확인한다.
4. 입이 화면의 중앙쯤에 왔는지 본다.

##### 아래 나타나는 입력박스에 'q'를 입력하면 비디오 플레이가 검사가 종료됩니다.

In [ ]:
    
for i, (name, clip) in enumerate(ppys.items()):
    print('i:', i)
    try:
        df = pd.read_csv(check_csv)
    except:
        df = pd.DataFrame([], columns=['clip', 'status'])
    
    if name in df['clip'].values:
        continue
    
    display(df.tail(3))
    
    cs = pd.read_pickle(clip/'df_fan.pickle')['cropped_size'].values[0]
    
    print(f'{(len(df), i)}/{len(ppys)}: name({name}), cropped_size({cs}px)')
    show_result.display_video(clip/'debug.mp4', width='width=600', playbackrate=1.5)
    c = input()
    if c == 'q':
        break
        
    checks = {'y':'yes', 'n':'no', 'ㅛ':'yes', 'ㅜ':'no'}
    if c in checks:
        status = checks[c]
    else:
        status = None 
        
    df = df.append({'clip':name, 'status': status}, ignore_index=True)
    
    df.to_csv(check_csv, index=False)
    ipd.clear_output(wait=True)